<a href="https://colab.research.google.com/github/edulapalle/CS584-ML-HW3/blob/main/ML_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#testing git hub